# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [1]:
import datetime
import pandas as pd

In [2]:
import sys
sys.path.append('../../')
%cd '../../'

/home/tsu-nera/repo/bakuchi


In [3]:
timestamp = "2003200210"

In [4]:
def read_trades(timestamp, exchange_id):
    file_path = "reports/{}/trades/{}.csv".format(timestamp, exchange_id)
    return pd.read_csv(file_path, index_col="id", parse_dates=["datetime"])

In [5]:
cc_trades = read_trades(timestamp, "coincheck")
lq_trades = read_trades(timestamp, "liquid")

In [6]:
len(cc_trades), len(lq_trades)

(311, 311)

In [7]:
def read_ticks(timestamp, exchange_id):
    file_path = "reports/{}/exchanges/{}.csv".format(timestamp, exchange_id)
    df = pd.read_csv(file_path, index_col="timestamp", parse_dates=["timestamp"])
    df["request_tick"] = df["tick_timestamp"]
    df["response_tick"] = df.index
    df = df.drop(["tick_timestamp"], axis=1)
    return df

In [8]:
cc_ticks = read_ticks(timestamp, "coincheck")
lq_ticks = read_ticks(timestamp, "liquid")

In [9]:
cc_ticks.head(5)

,bid,ask,request_tick,response_tick
timestamp,,,,
2020-03-20 02:10:49,688935.0,689702.0,2020-03-20 02:10:49,2020-03-20 02:10:49
2020-03-20 02:10:51,689474.0,689761.0,2020-03-20 02:10:51,2020-03-20 02:10:51
2020-03-20 02:10:52,689474.0,689761.0,2020-03-20 02:10:52,2020-03-20 02:10:52
2020-03-20 02:10:53,689474.0,689761.0,2020-03-20 02:10:53,2020-03-20 02:10:53
2020-03-20 02:10:54,689474.0,689761.0,2020-03-20 02:10:54,2020-03-20 02:10:54


In [10]:
trades = []
amount = 0.0065

for cc_row, lq_row in zip(cc_trades.iterrows(), lq_trades.iterrows()):
    cc_item = cc_row[1]
    lq_item = lq_row[1]
    
    cc_side = cc_item["side"]
    cc_timestamp = cc_item["datetime"]
    # cc_amount = cc_item["amount"]
    cc_actual_price = cc_item["price"]
    cc_actual_rate = cc_item["rate"]

    lq_side = lq_item["side"]
    lq_timestamp = lq_item["datetime"]
    # cc_amount = cc_item["amount"]
    lq_actual_price = lq_item["price"]
    lq_actual_rate = lq_item["rate"]
    
    try:
        cc_tick = cc_ticks.loc[cc_timestamp]
    except Exception:
        timestamp = cc_timestamp + datetime.timedelta(seconds=1)
        cc_tick = cc_ticks.loc[timestamp]
    try:
        lq_tick = lq_ticks.loc[lq_timestamp]
    except Exception:
        timestamp = lq_timestamp + datetime.timedelta(seconds=1)
        lq_tick = lq_ticks.loc[timestamp]        

    if cc_side == "sell":
        cc_tick_rate = cc_tick.bid
    else:
        cc_tick_rate = cc_tick.ask
    if lq_side == "sell":
        lq_tick_rate = lq_tick.bid
    else:
        lq_tick_rate = lq_tick.ask
        
    cc_expected_price = cc_tick_rate * amount
    lq_expected_price = lq_tick_rate * amount

    
    if cc_side == "buy":
        cc_expected_price = cc_expected_price * -1
        cc_actual_price = cc_actual_price * -1
    if lq_side == "buy":
        lq_expected_price = lq_expected_price * -1
        lq_actual_price = lq_actual_price * -1
        
    expected_profit = sum([cc_expected_price, lq_expected_price])
    actual_profit = sum([cc_actual_price, lq_actual_price])

    trade = {
        "timestamp": cc_timestamp, 
        "cc_side": cc_side,
        "cc_expected_rate": cc_tick_rate,
        "cc_actual_rate": cc_actual_rate,
        "cc_expected_price": cc_expected_price,
        "cc_actual_price": cc_actual_price,
        "lq_side": lq_side,
        "lq_expected_rate": lq_tick_rate,
        "lq_actual_rate": lq_actual_rate,
        "lq_expected_price": lq_expected_price,
        "lq_actual_price": lq_actual_price,
        "expected_profit": expected_profit,
        "actual_profit": actual_profit
    }
    
    trades.append(trade)

In [11]:
trade_df = pd.DataFrame(trades)
trade_df = trade_df.set_index("timestamp")

In [12]:
profit_df = trade_df[["expected_profit", "actual_profit"]]

In [20]:
int(sum(profit_df["expected_profit"])), int(sum(profit_df["actual_profit"]))

(1273, 644)

In [16]:
pd.set_option('display.max_rows', None)
profit_df

,expected_profit,actual_profit
timestamp,,
2020-03-20 02:10:49,13.1040,16.231
2020-03-20 02:30:22,-10.1920,-8.173
2020-03-20 02:30:31,17.8425,14.285
2020-03-20 02:32:43,-10.0620,-9.594
2020-03-20 02:32:48,14.2740,13.904
2020-03-20 02:46:25,-6.0905,-4.422
2020-03-20 02:46:45,15.2555,14.904
2020-03-20 02:46:53,-8.7360,-10.375
2020-03-20 02:46:59,17.4005,10.943
